In [1]:
#default_exp read
%load_ext autoreload
%autoreload 2

# Reading
> Este módulo concentra funções para a leitura dos diversos arquivos que compões a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.

In [ ]:
#export
from typing import Union
from pathlib import Path

import pandas as pd
from pyarrow import ArrowInvalid


from anateldb.format import df_optimize
from anateldb.query import update_mosaico, update_stel, update_radcom, update_base
from anateldb.merge import aero_common, aero_new


## MOSAICO
> O mosaico atualmente é composto por duas bases complementares: Estações e Plano Básico. Os módulos aqui leem, processam e mesclam essas bases para formar a base do Mosaico.

In [ ]:
#export
def read_mosaico(pasta: Union[str, Path], update: bool=False)->pd.DataFrame:
    """Lê o banco de dados salvo localmente do MOSAICO. 
    Opcionalmente o atualiza antes da leitura baixando os diversos arquivos disponíveis na interface web pública"""
    if update:
        update_mosaico(pasta)
    file = Path(f"{pasta}/mosaico.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/mosaico.xlsx")
        try:
            df = pd.read_excel(file)
        except FileNotFoundError:
            return read_mosaico(pasta, update=True)
    return df

## STEL & RADCOM

In [ ]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_stel(pasta)
    file = Path(f"{pasta}/stel.fth")
    try:
        stel = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/stel.xlsx")
        try:
            stel = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_stel(pasta, True)

    return stel


def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_radcom(pasta)
    file = Path(f"{pasta}/radcom.fth")
    try:
        radcom = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/radcom.xlsx")
        try:
            radcom = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_radcom(pasta, True)
    return radcom

## Bases Externas da Aeronáutica

In [ ]:
#export
def read_icao(pasta, update=False):
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    if update:
        # TODO: atualizar a base de dados do Frequency Finder e Canalização VOR/ILS/DME
        # update_icao(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/icao.fth")
    try:
        icao = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/icao.xlsx")
        try:
            icao = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return icao


def read_aisw(pasta, update=False):
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME
        # update_pmec(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/aisw.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/aisw.xlsx")
        try:
            df = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return df


def read_aisg(pasta, update=False):
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do GEOAISWEB
        # update_geo(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/aisg.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/aisg.xlsx")
        try:
            df = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return df

def read_aero(pasta, up_icao=False, up_pmec=False, up_geo=False):
    icao = read_icao(pasta, up_icao)
    pmec = read_aisw(pasta, up_pmec)
    geo = read_aisg(pasta, up_geo)
    icao["Description"] = icao.Description.astype("string")
    pmec["Description"] = pmec.Description.astype("string")
    geo["Description"] = geo.Description.astype("string")
    common = aero_common(icao, pmec, geo)
    new = aero_new(icao, pmec, geo)
    return common, new


## Base Consolidada

In [1]:
#export
def read_base(pasta, update=False):
    """Lê a base de dados e opcionalmente a atualiza antes da leitura"""
    if update:
        update_base(pasta)
    file = Path(f"{pasta}/base.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/base.xlsx")
        try:
            df = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            df = update_base(pasta, True)
    return df